In [4]:
#!pip install textblob
import pandas as pd
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer
import nltk
import langid

In [10]:
#!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /Users/anantnaw/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /Users/anantnaw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anantnaw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/anantnaw/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/anantnaw/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/anantnaw/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [90]:
tb = Blobber(analyzer=NaiveBayesAnalyzer())



In [143]:
Hotel_Reviews=pd.read_csv("Hotel_Reviews.csv", usecols=["Hotel_Name","Positive_Review","Negative_Review","Reviewer_Score"])


In [144]:
Hotel_Reviews_selected=Hotel_Reviews[:30000]


In [145]:
scoreSentContradictionSet=set()
categSentContradictionSet=set()
duplicateContradictionSet=set()

#othersets

In [88]:
Hotel_Reviews_selected.Negative_Review.loc[0]

' I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day SO i had to check out the next day before 11 o clock in order to get the room i waned to Not the best way to begin y

In [152]:
for i in Hotel_Reviews_selected.index:
    pos_sentiment=0
    neg_sentiment=0
    negative_review=''
    positive_review=''
    if (i%1000==0):
        print(i)
    if "No Negative" not in Hotel_Reviews_selected.Negative_Review[i]:
        negative_review=Hotel_Reviews_selected.Negative_Review[i]
        
    if "No Positive" not in Hotel_Reviews_selected.Positive_Review[i]:
        positive_review=Hotel_Reviews_selected.Positive_Review[i]
    if langid.classify(positive_review)[0]!='en' or langid.classify(negative_review)[0]!='en':
        continue
    pos_sent=tb(positive_review).sentiment[0]
    neg_sent=tb(negative_review).sentiment[0]
    if ((len(positive_review)>0 and pos_sent!='pos') or (len(negative_review)>0 and neg_sent!='neg')):
        categSentContradictionSet.add(i)
        
    #part 2  
    if(len(positive_review)==0 and len(negative_review)==0):
        continue
    score=Hotel_Reviews_selected.Reviewer_Score[i]
    polarity=tb(positive_review+' '+negative_review).sentiment[0]
    if ((Hotel_Reviews_selected.Reviewer_Score[i]<6 and polarity!='neg') or (Hotel_Reviews_selected.Reviewer_Score[i]>=9.0 and polarity!='pos')):
        scoreSentContradictionSet.add(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sps
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import langid
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anantnaw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def reviewWordList(reviews, wordLengthThreshold = 3, stopWordList = stopwords.words('English'), displayProgress = False):
    allWords = []
    i = 1
    for posRev in reviews:
        if i % ((len(reviews))/10) == 0 and displayProgress:
            print ('    ', 100 * (1.0 * i / len(reviews)), '% finished')
        i+=1
        allWords.append([f.split() for f in re.findall('\d+|\D+',posRev.lower())])

    allWords = list(itertools.chain.from_iterable(list(itertools.chain.from_iterable(allWords))))
    allWords = list(set(allWords))
    revWordLst = []
    for w in allWords:
        if len(w) > wordLengthThreshold and w not in stopWordList:
            revWordLst.append(w)
    return revWordLst

In [8]:
def getSimilarIndices(cosSim, simThreshold = .8, displayProgress = False):
    simRevs = []
    i = 1
    for index in range(cosSim.shape[0]):
        if i % (cosSim.shape[0]/10) == 0 and displayProgress:
            print ('    ', 100 * (1.0 * i / cosSim.shape[0]), '% finished')
        i+=1
        i_0 = np.array(cosSim[index].todense())[0]
        sims = []
        for where in np.argwhere(i_0 > simThreshold):
            if where[0] != index:
                #print where[0], lAPR[where[0]]
                sims.append(where[0])
        simRevs.append([index, sims])
    return simRevs

In [153]:
def getSimilarTexts(reviews, simThreshold = .85, displayProgress = False, wordLengthThreshold = 3, stopWordList = stopwords.words('English')):
    if displayProgress:
        print("Extracting Important Texts")
    rwl = reviewWordList(reviews, wordLengthThreshold = 3, stopWordList = stopwords.words('English'), displayProgress=displayProgress)
    
    if displayProgress:
        print("Constructing TF-IDF Vectorizer")
    vectorizer = TfidfVectorizer(stop_words=stopwords.words('English'), vocabulary=rwl)
    
    if displayProgress:
        print("Constructing Feature Matrix")
    featureMatrix = vectorizer.fit_transform(reviews)
    
    if displayProgress:
        print("Constructing Cosine Similarity Matrix")
    cosSim = cosine_similarity(featureMatrix, Y=None, dense_output=False)
    
    if displayProgress:
        print("Extracting Similar Reviews")
    simRevs = getSimilarIndices(cosSim, simThreshold=simThreshold, displayProgress=displayProgress)
    
    return simRevs

In [154]:

print(len(categSentContradictionSet),' ', len(scoreSentContradictionSet),' ',len(categSentContradictionSet.intersection(scoreSentContradictionSet)))

14890   3976   2888


In [6]:
print(Hotel_Reviews.Positive_Review[:9])

0     Only the park outside of the hotel was beauti...
1     No real complaints the hotel was great great ...
2     Location was good and staff were ok It is cut...
3     Great location in nice surroundings the bar a...
4      Amazing location and building Romantic setting 
5     Good restaurant with modern design great chil...
6     The room is spacious and bright The hotel is ...
7     Good location Set in a lovely park friendly s...
8                                          No Positive
Name: Positive_Review, dtype: object


In [147]:
Hotel_Reviews_selected_with_Positives=Hotel_Reviews_selected[Hotel_Reviews_selected.Positive_Review!="No Positive"]

In [148]:
similarity_indices=getSimilarTexts(Hotel_Reviews_selected_with_Positives.Positive_Review)

In [138]:
similarity_indices

[[0, []],
 [1, []],
 [2, []],
 [3, []],
 [4, []],
 [5, []],
 [6, []],
 [7, []],
 [8, []],
 [9, []],
 [10, [1495, 9640, 21118]],
 [11, [2871, 2923, 9730, 11166, 12658, 13251, 15409, 23549, 25996]],
 [12, []],
 [13, []],
 [14, []],
 [15, []],
 [16, []],
 [17, []],
 [18, []],
 [19, []],
 [20, []],
 [21, []],
 [22, []],
 [23, []],
 [24, []],
 [25, []],
 [26, []],
 [27, []],
 [28, []],
 [29, []],
 [30, []],
 [31, []],
 [32, []],
 [33, []],
 [34, []],
 [35, []],
 [36, []],
 [37, []],
 [38, []],
 [39, []],
 [40, []],
 [41, []],
 [42, []],
 [43, []],
 [44, []],
 [45, []],
 [46, []],
 [47,
  [1692,
   2518,
   2906,
   3015,
   6500,
   6849,
   7925,
   8348,
   9238,
   9896,
   10100,
   10698,
   10853,
   10965,
   15585,
   15816,
   16006,
   16996,
   17206,
   19889,
   20600,
   21353,
   23362,
   23947,
   24311,
   28610,
   29985]],
 [48, []],
 [49, [6452, 15759, 15860, 17985, 21320, 22101, 27059, 27158, 28727]],
 [50, []],
 [51, []],
 [52, []],
 [53, []],
 [54, []],
 [55, []],
 [

In [69]:
print(Hotel_Reviews_selected.loc[32356])

Hotel_Name                            Shepherd s Bush Boutique Hotel
Negative_Review     Some reception staff very helpful but some la...
Positive_Review     Good location close to tube station and Westf...
Reviewer_Score                                                   7.5
Name: 32356, dtype: object


In [149]:
for item in similarity_indices:
    for item2 in item[1]:
         duplicateContradictionSet.add(item2)
    

    
    
    

In [150]:
dupliset=set()
contraList=list(duplicateContradictionSet)
for item in contraList:
       dupliset.add(Hotel_Reviews_selected_with_Positives.iloc[item].name)

In [151]:
print(len(categSentContradictionSet),' ', len(scoreSentContradictionSet),' ',len(categSentContradictionSet.intersection(scoreSentContradictionSet)))

7685

In [155]:
print(len(categSentContradictionSet),' ', len(scoreSentContradictionSet),' ',len(categSentContradictionSet.intersection(scoreSentContradictionSet).intersection(dupliset)))

14890   3976   942
